In [ ]:
!pip install -q streamlit
!pip install python-docx
!pip install reportlab
!pip install docx2txt
!pip install pdfplumber
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 1s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠇

In [ ]:
%%writefile app.py

import os
import io
import json
import tempfile
from typing import Dict, Any, List

import streamlit as st
import openai
import docx
import docx2txt
import pdfplumber
from docx import Document
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont


# ---------------------------
# Helpers: OpenAI interaction
# ---------------------------
client = openai.OpenAI(
    base_url="https://aiportalapi.stu-platform.live/jpe",
    api_key="")

pdfmetrics.registerFont(TTFont('DejaVu', 'DejaVuSans.ttf'))
def build_system_prompt():
    return (
        "Bạn là chuyên gia nhân sự ngành CNTT và chuyên viên viết CV chuyên nghiệp. "
        "Nhiệm vụ: nhận dữ liệu người dùng (thông tin cá nhân, kỹ năng, kinh nghiệm, dự án) "
        "và/hoặc JD. Phân tích sự phù hợp giữa hồ sơ và JD, gợi ý cải thiện, "
        "và tạo CV hoàn chỉnh, hấp dẫn, chuẩn ngành CNTT."
    )


def call_openai_generate_cv(user_profile: Dict[str, Any], job_description: Dict[str, Any]):
    system_msg = build_system_prompt()

    few_shot_example = (
        "Ví dụ:\n"
        "Input:\nNgười dùng có kỹ năng Python, Django, REST API; kinh nghiệm 2 năm backend.\n"
        "JD: Python Developer yêu cầu Django, REST API, PostgreSQL.\n\n"
        "Output:\n"
        "Tóm tắt: Lập trình viên Python có 2 năm kinh nghiệm phát triển API.\n"
        "Kỹ năng: Python, Django, REST API, PostgreSQL.\n"
        "Kinh nghiệm: Tối ưu API nội bộ giúp tăng 30% tốc độ xử lý.\n"
        "=> Hãy tạo CV hoàn chỉnh tương tự với dữ liệu người dùng cung cấp."
    )

    user_payload = {
        "user_profile": user_profile,
        "job_description": job_description
    }

    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": few_shot_example},
        {"role": "user", "content": json.dumps(user_payload, ensure_ascii=False, indent=2)}
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.2,
            max_tokens=1000
        )
        return response.choices[0].message.content
    except Exception as e:
        st.error(f"Lỗi OpenAI API: {e}")
        return ""


# ---------------------------
# Helpers: document export
# ---------------------------

def create_docx_from_text(text: str) -> bytes:
    doc = Document()
    for para in text.split("\n\n"):
        para = para.strip()
        if not para:
            continue
        doc.add_paragraph(para)
    bio = io.BytesIO()
    doc.save(bio)
    bio.seek(0)
    return bio.read()


def create_pdf_from_text(text: str) -> bytes:
    bio = io.BytesIO()
    c = canvas.Canvas(bio, pagesize=A4)
    width, height = A4
    margin = 40
    y = height - margin

    c.setFont("DejaVu", 12)

    lines = []
    for paragraph in text.split("\n\n"):
        paragraph = paragraph.strip()
        if not paragraph:
            lines.append("")
            continue
        words = paragraph.split()
        cur = ""
        for w in words:
            trial = cur + (" " if cur else "") + w
            if len(trial) > 90:
                lines.append(cur)
                cur = w
            else:
                cur = trial
        if cur:
            lines.append(cur)
        lines.append("")

    for line in lines:
        if y < margin + 14:
            c.showPage()
            c.setFont("DejaVu", 11)
            y = height - margin
        c.drawString(margin, y, line)
        y -= 14

    c.save()
    bio.seek(0)
    return bio.read()


# ---------------------------
# Extract text from uploads
# ---------------------------

def extract_text_from_upload(uploaded_file) -> str:
    if uploaded_file is None:
        return ""
    name = uploaded_file.name.lower()
    content = uploaded_file.read()
    if name.endswith(".txt"):
        return content.decode("utf-8", errors="ignore")
    elif name.endswith(".docx"):
        with tempfile.NamedTemporaryFile(delete=False, suffix=".docx") as tmp:
            tmp.write(content)
            tmp.flush()
            return docx2txt.process(tmp.name) or ""
    elif name.endswith(".pdf"):
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
            tmp.write(content)
            tmp.flush()
            text = ""
            with pdfplumber.open(tmp.name) as pdf:
                for page in pdf.pages:
                    text += page.extract_text() + "\n"
            return text.strip()
    else:
        return content.decode("utf-8", errors="ignore")


# ---------------------------
# Streamlit UI
# ---------------------------

st.set_page_config(page_title="AI CV Builder - IT", layout="centered")
st.title("AI CV Builder - Tạo và cải thiện CV ngành CNTT")


st.markdown("Nhập thông tin hoặc tải lên CV/JD để tạo hoặc cải thiện CV chuyên nghiệp trong ngành CNTT.")

with st.expander("Thông tin cá nhân"):
    full_name = st.text_input("Họ và tên", "Nguyễn Văn A")
    email = st.text_input("Email", "nguyenvana@example.com")
    phone = st.text_input("Số điện thoại", "09xxxxxxxx")
    education = st.text_input("Học vấn", "Đại học CNTT - Khoa CNTT")
    skills_input = st.text_area("Kỹ năng (ngăn cách bằng dấu phẩy)", "Python, Django, REST API, PostgreSQL")
    skills = [s.strip() for s in skills_input.split(",") if s.strip()]

    exp_raw = st.text_area("Kinh nghiệm (mỗi dòng: vị trí | công ty | năm | mô tả)",
                           "Backend Developer | ABC Tech | 2 | Xây dựng REST API giúp tăng tốc xử lý dữ liệu")
    experiences = []
    for line in exp_raw.splitlines():
        if not line.strip():
            continue
        parts = [p.strip() for p in line.split("|")]
        exp = {"position": parts[0], "company": parts[1], "years": parts[2], "achievements": parts[3] if len(parts) > 3 else ""}
        experiences.append(exp)

    projects_raw = st.text_area("Dự án (mỗi dòng: tên | công nghệ | mô tả)",
                                "Hệ thống quản lý kho | Python, PostgreSQL | Quản lý tồn kho")
    projects = []
    for line in projects_raw.splitlines():
        if not line.strip():
            continue
        parts = [p.strip() for p in line.split("|")]
        projects.append({"name": parts[0], "tech_stack": parts[1], "description": parts[2] if len(parts) > 2 else ""})

with st.expander("JD hoặc CV Upload"):
    jd_text = st.text_area("Job Description (JD)", "Python Developer cần Django, REST API, PostgreSQL, ưu tiên Docker.")
    uploaded_cv = st.file_uploader("Upload CV (.txt, .docx, .pdf)", type=["txt", "docx", "pdf"])
    uploaded_cv_text = extract_text_from_upload(uploaded_cv) if uploaded_cv else ""

st.markdown("---")

col1, col2, col3 = st.columns([1, 1, 1])
generate_cv = col1.button("Tạo CV từ Form")
improve_jd = col2.button("Cải thiện CV theo JD")
improve_upload = col3.button("Cải thiện CV từ File")

user_profile = {
    "full_name": full_name,
    "email": email,
    "phone": phone,
    "education": education,
    "skills": skills,
    "experience": experiences,
    "projects": projects
}
job_description = {"raw_text": jd_text}

generated_cv_text = ""

if generate_cv:
        st.info("Đang tạo CV, vui lòng chờ...")
        generated_cv_text = call_openai_generate_cv(user_profile, {"raw_text": ""})
elif improve_jd:
        st.info("Đang cải thiện CV theo JD, vui lòng chờ...")
        generated_cv_text = call_openai_generate_cv(user_profile, job_description)
elif improve_upload:
        if not uploaded_cv_text:
            st.warning("Vui lòng tải lên file CV trước.")
        else:
            st.info("Đang cải thiện CV từ file upload...")
            generated_cv_text = call_openai_generate_cv(user_profile, {"raw_text": uploaded_cv_text + "\n\nJD:\n" + jd_text})

if generated_cv_text:
    st.success("CV đã sẵn sàng.")
    edited_text = st.text_area("Xem và chỉnh sửa CV tại đây:", value=generated_cv_text, height=400, key="cv_preview")

    colw, colp = st.columns(2)
    with colw:
        docx_bytes = create_docx_from_text(edited_text)
        st.download_button("Tải xuống Word (.docx)", data=docx_bytes,
                           file_name=f"{full_name.replace(' ', '_')}_CV.docx",
                           mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document")

    with colp:
        pdf_bytes = create_pdf_from_text(edited_text)
        st.download_button("Tải xuống PDF (.pdf)", data=pdf_bytes,
                           file_name=f"{full_name.replace(' ', '_')}_CV.pdf",
                           mime="application/pdf")

st.markdown("---")
st.caption("Developed with Streamlit and OpenAI API")


Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl https://loca.lt/mytunnelpassword

104.197.237.38⠙⠹your url is: https://short-cities-love.loca.lt
